**[CNN-01]** 必要なモジュールをインポートして、乱数のシードを設定します。

In [1]:
# Chapter05で最初に実行
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

# jupyter notebook上にグラフを表示するために必要
%matplotlib inline

np.random.seed(20160704)
tf.set_random_seed(20160704)

**[CNN-02]** MNISTのデータセットを用意します。

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[CNN-03]** 1段目の畳み込みフィルターとプーリング層を定義します。

In [3]:
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],
                                          stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                       strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

**[CNN-04]** 2段目の畳み込みフィルターとプーリング層を定義します。

In [4]:
num_filters2 = 64

W_conv2 = tf.Variable(
            tf.truncated_normal([5,5,num_filters1,num_filters2],
                                stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

**[CNN-05]** 全結合層、ドロップアウト層、ソフトマックス関数を定義します。

In [5]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

**[CNN-06]** 誤差関数 loss、トレーニングアルゴリズム train_step、正解率 accuracy を定義します。

In [6]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**[CNN-07]** セッションを用意して、Variable を初期化します。

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

**[CNN-08]** パラメーターの最適化を20000回繰り返します。

最終的に、テストセットに対して約99%の正解率が得られます。

In [8]:
i = 0
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = len(mnist.test.labels) // 4 * c
            end = len(mnist.test.labels) // 4 * (c+1)
            loss_val, acc_val = sess.run([loss, accuracy],
                feed_dict={x:mnist.test.images[start:end],
                           t:mnist.test.labels[start:end],
                           keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        saver.save(sess, 'cnn_session', global_step=i)

Step: 500, Loss: 1541.379883, Accuracy: 0.955900
Step: 1000, Loss: 971.882324, Accuracy: 0.971300
Step: 1500, Loss: 802.896118, Accuracy: 0.973300
Step: 2000, Loss: 644.275146, Accuracy: 0.978900
Step: 2500, Loss: 604.005737, Accuracy: 0.981200
Step: 3000, Loss: 561.430176, Accuracy: 0.982000
Step: 3500, Loss: 463.946960, Accuracy: 0.985200
Step: 4000, Loss: 429.951569, Accuracy: 0.986700
Step: 4500, Loss: 406.654999, Accuracy: 0.986500
Step: 5000, Loss: 403.234894, Accuracy: 0.987200
Step: 5500, Loss: 367.896606, Accuracy: 0.988800
Step: 6000, Loss: 384.979156, Accuracy: 0.987300
Step: 6500, Loss: 353.671204, Accuracy: 0.988200
Step: 7000, Loss: 361.740540, Accuracy: 0.988700
Step: 7500, Loss: 334.247467, Accuracy: 0.988900
Step: 8000, Loss: 355.330322, Accuracy: 0.988400
Step: 8500, Loss: 348.331116, Accuracy: 0.988000
Step: 9000, Loss: 306.451538, Accuracy: 0.990400
Step: 9500, Loss: 330.099854, Accuracy: 0.989000
Step: 10000, Loss: 303.053223, Accuracy: 0.990900
Step: 10500, Loss: 

**[CNN-09]** セッション情報を保存したファイルが生成されていることを確認します。

In [9]:
!ls cnn_session*

cnn_session-1000.data-00000-of-00001   cnn_session-19000.data-00000-of-00001
cnn_session-1000.index		       cnn_session-19000.index
cnn_session-1000.meta		       cnn_session-19000.meta
cnn_session-10000.data-00000-of-00001  cnn_session-19500.data-00000-of-00001
cnn_session-10000.index		       cnn_session-19500.index
cnn_session-10000.meta		       cnn_session-19500.meta
cnn_session-10500.data-00000-of-00001  cnn_session-2000.data-00000-of-00001
cnn_session-10500.index		       cnn_session-2000.index
cnn_session-10500.meta		       cnn_session-2000.meta
cnn_session-11000.data-00000-of-00001  cnn_session-20000.data-00000-of-00001
cnn_session-11000.index		       cnn_session-20000.index
cnn_session-11000.meta		       cnn_session-20000.meta
cnn_session-11500.data-00000-of-00001  cnn_session-2500.data-00000-of-00001
cnn_session-11500.index		       cnn_session-2500.index
cnn_session-11500.meta		       cnn_session-2500.meta
cnn_session-12000.data-00000-of-00001  cnn_session-3000.data-00000-of-000